In [97]:
import pandas as pd
import table_cleaning_functions as tcf

In [98]:
# load raw data 
raw_df = pd.read_csv('../home/raw_data/clinvar_biomarkers_revised.csv')
raw_df = raw_df.drop(raw_df.columns[0], axis=1)
raw_df.head()

,biomarker_ID,main_Xref,assessed_biomarker_entity,biomarker_status,best_biomarker_type,specimen_type,loinc_code,condition_name,assessed_entity_type,evidence_source,notes,rsID,disease,uniprot,Name,SYSTEM,Gene,DOID,mutation,variation
0,NaN,dbSNP:rs1800562,rs1800562 mutation in hereditary haemochromato...,presence of,risk_biomarker,blood (UN:0000178),48579-7,cancer (DOID:162),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9UK37 (NM_000410.4(HFE):c.845G>A (p.Cys2...,1800562.0,cancer,Q9UK37,Hereditary haemochromatosis protein (Hereditar...,Bld/Tiss,HFE,162.0,NM_000410.4(HFE):c.845G>A (p.Cys282Tyr),9.0
1,NaN,dbSNP:rs121908830,rs121908830 mutation in transmembrane protein ...,presence of,risk_biomarker,NaN,NaN,cancer (DOID:162),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:C9J4H2 (NM_017849.4(TMEM127):c.475C>T (p....,121908830.0,cancer,C9J4H2,Transmembrane protein 127,NaN,TMEM127,162.0,NM_017849.4(TMEM127):c.475C>T (p.Gln159Ter),108.0
2,NaN,dbSNP:rs137854550,rs137854550 mutation in neurofibromin (NF1),presence of,risk_biomarker,amniotic fluid (UN:0000173),45331-6,cancer (DOID:162),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9UMU3 (NM_001042492.3(NF1):c.4330A>C (p....,137854550.0,cancer,Q9UMU3,Neurofibromin,Amnio fld,NF1,162.0,NM_001042492.3(NF1):c.4330A>C (p.Lys1444Gln),431637.0
3,NaN,dbSNP:rs137854552,rs137854552 mutation in neurofibromin (NF1),presence of,risk_biomarker,amniotic fluid (UN:0000173),45331-6,cancer (DOID:162),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9UMU3 (NM_001042492.3(NF1):c.5902C>A (p....,137854552.0,cancer,Q9UMU3,Neurofibromin,Amnio fld,NF1,162.0,NM_001042492.3(NF1):c.5902C>A (p.Arg1968=),825999.0
4,NaN,dbSNP:rs137854560,rs137854560 mutation in neurofibromin (NF1),presence of,risk_biomarker,amniotic fluid (UN:0000173),45331-6,cancer (DOID:162),gene,ClinVar|https://www.ncbi.nlm.nih.gov/clinvar/v...,UPKB:Q9UMU3 (NM_001042492.3(NF1):c.4084C>T (p....,137854560.0,cancer,Q9UMU3,Neurofibromin,Amnio fld,NF1,162.0,NM_001042492.3(NF1):c.4084C>T (p.Arg1362Ter),344.0


In [99]:
for col in raw_df.columns:
    raw_df[col] = tcf.strip_values(raw_df[col])

In [100]:
# check for missing values
raw_df.isnull().sum()

biomarker_ID                 151469
main_Xref                      1232
assessed_biomarker_entity         0
biomarker_status                  0
best_biomarker_type               0
specimen_type                 35933
loinc_code                    35933
condition_name                    0
assessed_entity_type              0
evidence_source                   1
notes                             1
rsID                           1233
disease                           1
uniprot                         253
Name                            253
SYSTEM                        35933
Gene                              1
DOID                            868
mutation                       1233
variation                      1233
dtype: int64

In [101]:
raw_df.shape

(151469, 20)

In [102]:
# drop rows with missing evidence_source or RSID values 
raw_df = raw_df.dropna(subset=['evidence_source'])
raw_df = raw_df.dropna(subset=['rsID'])
raw_df.shape

(150236, 20)

In [103]:
new_df = pd.DataFrame(columns = tcf.TSV_HEADERS)
new_df.head()

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag


In [104]:
new_df['biomarker'] = raw_df['biomarker_status'] + ' ' + raw_df['assessed_biomarker_entity']
new_df['assessed_biomarker_entity_id'] = raw_df['main_Xref']
new_df['assessed_biomarker_entity'] = raw_df['Gene']
new_df['assessed_entity_type'] = raw_df['assessed_entity_type']
new_df['best_biomarker_role'] = raw_df['best_biomarker_type'].str.replace('_biomarker', '')
new_df['specimen'] = raw_df['specimen_type'].str.findall(r'^(.*?)\(').str[0]
new_df['specimen_id'] = raw_df['specimen_type'].str.findall(r'\((.*?)\)$').str[0]
new_df['loinc_code'] = raw_df['loinc_code']
new_df['condition'] = raw_df['condition_name'].str.findall(r'^(.*?)\(').str[0] 
new_df['condition_id'] = raw_df['condition_name'].str.findall(r'\((.*?)\)$').str[0]
raw_df['evidence_source'] = raw_df['evidence_source'].fillna('')
new_df['evidence_source'] = 'ClinVar:' + raw_df['evidence_source'].str.findall(r'variation/(\d+)').str[0]
new_df['evidence'] = ''
new_df['tag'] = 'biomarker;assessed_biomarker_entity_id;assessed_biomarker_entity;specimen;condition'
new_df.head()

,biomarker_id,biomarker,assessed_biomarker_entity,assessed_biomarker_entity_id,assessed_entity_type,condition,condition_id,exposure_agent,exposure_agent_id,best_biomarker_role,specimen,specimen_id,loinc_code,evidence_source,evidence,tag
0,NaN,presence of rs1800562 mutation in hereditary h...,HFE,dbSNP:rs1800562,gene,cancer,DOID:162,NaN,NaN,risk,blood,UN:0000178,48579-7,ClinVar:9,,biomarker;assessed_biomarker_entity_id;assesse...
1,NaN,presence of rs121908830 mutation in transmembr...,TMEM127,dbSNP:rs121908830,gene,cancer,DOID:162,NaN,NaN,risk,NaN,NaN,NaN,ClinVar:108,,biomarker;assessed_biomarker_entity_id;assesse...
2,NaN,presence of rs137854550 mutation in neurofibro...,NF1,dbSNP:rs137854550,gene,cancer,DOID:162,NaN,NaN,risk,amniotic fluid,UN:0000173,45331-6,ClinVar:431637,,biomarker;assessed_biomarker_entity_id;assesse...
3,NaN,presence of rs137854552 mutation in neurofibro...,NF1,dbSNP:rs137854552,gene,cancer,DOID:162,NaN,NaN,risk,amniotic fluid,UN:0000173,45331-6,ClinVar:825999,,biomarker;assessed_biomarker_entity_id;assesse...
4,NaN,presence of rs137854560 mutation in neurofibro...,NF1,dbSNP:rs137854560,gene,cancer,DOID:162,NaN,NaN,risk,amniotic fluid,UN:0000173,45331-6,ClinVar:344,,biomarker;assessed_biomarker_entity_id;assesse...


In [105]:
new_df.shape

(150236, 16)

In [106]:
new_df.isnull().sum()

biomarker_id                    150236
biomarker                            0
assessed_biomarker_entity            0
assessed_biomarker_entity_id         0
assessed_entity_type                 0
condition                          858
condition_id                       858
exposure_agent                  150236
exposure_agent_id               150236
best_biomarker_role                  0
specimen                         35678
specimen_id                      35678
loinc_code                       35678
evidence_source                      0
evidence                             0
tag                                  0
dtype: int64

In [107]:
# drop null condition rows 
new_df = new_df.dropna(subset=['condition'])
new_df.shape

(149378, 16)

In [108]:
new_df.isnull().sum()

biomarker_id                    149378
biomarker                            0
assessed_biomarker_entity            0
assessed_biomarker_entity_id         0
assessed_entity_type                 0
condition                            0
condition_id                         0
exposure_agent                  149378
exposure_agent_id               149378
best_biomarker_role                  0
specimen                         35038
specimen_id                      35038
loinc_code                       35038
evidence_source                      0
evidence                             0
tag                                  0
dtype: int64

In [111]:
new_df['biomarker'] = tcf.clean_parantheticals(new_df['biomarker'])
new_df['specimen_id'] = tcf.map_prefixes(new_df['specimen_id'])

In [112]:
new_df.to_csv('../home/cleaned_data/clinvar_biomarkers.tsv', sep='\t', index=False)